# Net Demand

`Net Demand` = `Total Demand` - `Vesting Contract`
             = `Demand` + `TCL` + `Transmisson Loss` - `Vesting Contract`

In [1]:
import pandas as pd

rt_dpr = pd.read_csv('./data/RT_DPR.csv')
rt_dpr = rt_dpr[['Date', 'Period', 'Demand', 'TCL', 'TransmissionLoss']]
rt_dpr['TransmissionLoss'] = rt_dpr['TransmissionLoss'].fillna(0)
rt_dpr = rt_dpr[rt_dpr['Date'] > '2023-06-30']
rt_dpr = rt_dpr.sort_values(by=['Date', 'Period'])
rt_dpr.reset_index(drop=True, inplace=True)
rt_dpr.head(1)

,Date,Period,Demand,TCL,TransmissionLoss
0,2023-07-01,1,6086.883,0.0,32.68


In [3]:
vc_per = pd.read_csv('./data/VCData_Period.csv')

Create a view of [Total Demand, TCQ, Net Demand] from `rt_dpr` and `vc_per`.

In [18]:
rt_dpr['Total Demand'] = rt_dpr['Demand'] + rt_dpr['TCL'] + rt_dpr['TransmissionLoss']
view = rt_dpr[['Date', 'Period', 'Total Demand']].copy()

In [6]:
import holidays

sg_holidays = holidays.country_holidays('SG')

In [19]:
import datetime as dt
def find_tcq(row):
    # print(row)
    date_obj = dt.datetime.strptime(row['Date'], '%Y-%m-%d')
    year = date_obj.year
    quarter = (date_obj.month - 1) // 3 + 1
    
    isWeekend = 1 if date_obj.isoweekday() > 5 else 0
    isPublicHoliday = date_obj in sg_holidays
    
    if isWeekend or isPublicHoliday:
        return vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter)]['TCQ_Weekend_PH'].values[0] / 1000
    else:
        return vc_per[(vc_per['Year'] == year) & (vc_per['Quarter'] == quarter)]['TCQ_Weekday'].values[0] / 1000

In [20]:
view['TCQ'] = view.apply(lambda row: find_tcq(row), axis=1)
view['Net Demand'] = view['Total Demand'] - view['TCQ']
view.head(10)

,Date,Period,Total Demand,TCQ,Net Demand
0,2023-07-01,1,6119.563,486.146668,5633.416332
1,2023-07-01,2,6019.671,486.146668,5533.524332
2,2023-07-01,3,5910.691,486.146668,5424.544332
3,2023-07-01,4,5843.399,486.146668,5357.252332
4,2023-07-01,5,5797.707,486.146668,5311.560332
5,2023-07-01,6,5747.641,486.146668,5261.494332
6,2023-07-01,7,5697.556,486.146668,5211.409332
7,2023-07-01,8,5635.452,486.146668,5149.305332
8,2023-07-01,9,5582.066,486.146668,5095.919332
9,2023-07-01,10,5556.647,486.146668,5070.500332


In [9]:
view.to_csv('./data/NET.csv', index=False)

In [21]:
# Split dataset into Weekday dataset + Weekend/PH dataset
# Returns 1 if Weekend or PH, 0 is weekday
def type_of_day(row):
    # print(row)
    date_obj = dt.datetime.strptime(row['Date'], '%Y-%m-%d')
    year = date_obj.year
    quarter = (date_obj.month - 1) // 3 + 1
    
    isWeekend = 1 if date_obj.isoweekday() > 5 else 0
    isPublicHoliday = date_obj in sg_holidays
    
    if isWeekend or isPublicHoliday:
        return 1
    else:
        return 0

In [33]:
view['Type_Of_Day'] = view.apply(lambda row: type_of_day(row), axis=1)

df = view.copy()
weekend_ph_df = df[df['Type_Of_Day'] == 1]
weekday_df = df[df['Type_Of_Day'] == 0]
weekend_ph_df.to_csv('./data/Weekend_Demand.csv', index=False)
weekday_df.to_csv('./data/Weekday_Demand.csv', index=False)
